# **Data Retrieval**

In [1]:
import os 
import time
import seaborn as sns
import pandas as pd
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import pprint


In [2]:
# '''
# user-read-private \
# user-read-email \
# user-read-playback-state \
# user-library-read \
# playlist-modify-public \
# playlist-read-private \
# user-follow-read \
# user-top-read streaming
# '''

###ADD OAuth procedure back in so you can give users the option of saving the recommended tracks. 

Create application from spotify developers site and retrieve Client ID and Secret. Set up Spotify Client Credentials and spotipy object. 

In [3]:
CLIENT_ID = os.environ.get("SPOTIFY_CLIENT_ID")
SECRET = os.environ.get("SPOTIFY_SECRET")

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id = CLIENT_ID, client_secret = SECRET)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


In [5]:
#List of genres accepted by recommendation endpoint
FULL_GENRE_SEED_LST = sp.recommendation_genre_seeds()['genres']

fl = u'\u266D' #flat symbol
sh = u'\u266F' #sharp symbol

#Mapping For Tonics
music_keys = {
    -1: "No key detected {}".format(u'\u2639'),
    0: "C",
    1: "C{}/D{}".format(sh,fl),
    2: "D",
    3: "D{}/E{}".format(sh,fl),
    4: "E",
    5: "F",
    6: "F{}/G{}".format(sh,fl),
    7: "G",
    8: "G/A".format(sh,fl),
    9: "A",    
    10: "A{}/B{}".format(sh,fl),
    11: "B"
}

#Mapping For Modes
mode = {0: 'Min', 1: "Maj"}


In [6]:
#for queries in search call to API, use %20 or + for spaces in parameter q. required params
#are q for the query, type (artist, album, track, playlist, episode)
#nonrequired is limit, should use (default is 20, max 50, min 1).
first_res = sp.search("track:immigrant song artist:led zeppelin", limit=2)


- 100% should use caching on this. Will make future searches much faster. Find way to update/clear cache monthly.
- Probably will have to add conditional statement to also search for the artist if the user specifies they want to search that way. Possible options could be search for related tracks in same genre, tracks of related artists, or both. 

# Setting variables from initial search. Place in function

In [7]:
# def set_search_track_values(t,a):
#     search_d = {}
#     res = sp.search("track:{} artist:{}",limit=1).format(t.lower().strip(), a.lower().strip())
#     while res[]==:
#         new = input("This track was not found. Please try another: ")
#         res = sp.search("track:{} artist:{}",limit=1).format(t.lower().strip(), a.lower().strip())
#     search_d['search_track_name'] = first_res['tracks']['items'][0]['name']
#     search_d['search_track_uri'] = first_res['tracks']['items'][0]['uri'] #.split(":")[-1]
#     search_d['search_arts_uri'] = [x['uri'] for x in first_res['tracks']['items'][0]['artists']]
#     search_d['search_album_uri'] = first_res['tracks']['items'][0]['album']['uri']
#     search_d['search_alb_type'] = first_res['tracks']['items'][0]['album']['album_type']
#     search_d['search_alb_cov'] = first_res['tracks']['items'][0]['album']['images'][1]['url']
#     search_d['search_popularity'] = first_res['tracks']['items'][0]['popularity']
#     return search_d

        
prim_track_uri = first_res['tracks']['items'][0]['uri'] #.split(":")[-1]
art_uri_lst = [x['uri'] for x in first_res['tracks']['items'][0]['artists']]
album_uri = first_res['tracks']['items'][0]['album']['uri']
alb_type = first_res['tracks']['items'][0]['album']['album_type']
prim_alb_cov = first_res['tracks']['items'][0]['album']['images'][1]['url']
prim_popularity = first_res['tracks']['items'][0]['popularity']
prim_track_name = first_res['tracks']['items'][0]['name']


# Retrive Audio Features From Search Track

In [91]:
def get_search_aud_feats(uri):
    search_aud_feats = sp.audio_features(uri)
    return search_aud_feats[0]

def display_aud_feats(d):
    search_track_key = music_keys[d['key']] + " " + mode[d['mode']]
    search_tempo = round(d['tempo'])
    search_art = d['search_alb_cov']
    

### Grab genres from the original search artists call to try and additionally use as seeds for recommendation.

In [92]:
def get_genres(d):
    genre_lst = sp.artists(d['search_arts_uri'])['artists'][0]['genres']
    genre_seeds = [x for x in genre_lst if x in FULL_GENRE_SEED_LST]
    return genre_seeds if len(genre_seeds) >= 1 else None


### Get the top tracks from the primary search artist(s) to use as seeds for recommendation.

In [11]:
#RETHINK THIS. WHY ARE TOP TRACKS OF AN ARTIST GOING TO BE GOOD SEEDS FOR A PARTICULAR SONG OF THEIRS??
#WILL ONLY NEED 4 ADDITIONAL TRACKS SINCE 1 WILL BE THE ORIGINAL SEARCH SONG
def get_tracks(d):
    #Retrieves top tracks for either the only artist listed on song or first two artists listed on song
    if len(d['search_arts_uri']) > 1:
        first_art_tracks = sp.artist_top_tracks(d['search_arts_uri'][0])
        first_art_tracks_uri = [x['uri'] for x in first_art_tracks['tracks'][:2]]
        sec_art_tracks = sp.artist_top_tracks(d['search_arts_uri'][0])
        
        #Append artists' top two tracks and searched song into single list for seed parameter use. 
        multi_art_tracks_uri = [first_art_track_uris.append(x['uri']) for x in sec_art_tracks['tracks'][:2]].append(d['search_track_uri'])
        return multi_art_tracks_uri
    else:
        art_tracks = sp.artist_top_tracks(d['search_arts_uri'][0])
        single_art_tracks_uri = [x['uri'] for x in art_tracks['tracks'][:4]].append(d['search_track_uri'])
        return single_art_tracks_uri

dt = {'search_arts_uri': art_uri_lst[0], 'search_track_uri': prim_track_uri}


### Call for related artists to use as seeds for the API's recommendation endpoint.

In [12]:
def get_rel_arts(artist_uris):
    if len(artist_uris) > 4:
        return
    else:
        rel_art_call_data = sp.artist_related_artists(artist_uris[0])
        for x in rel_art_call_data['artists']:
            if len(artist_uris) < 5:
                artist_uris.append(x['uri'])
            else:
                break
    return artist_uris


In [60]:
aud_feats_prim = sp.audio_features(prim_track_uri)
print(aud_feats_prim)

[{'danceability': 0.831, 'energy': 0.794, 'key': 2, 'loudness': -3.701, 'mode': 0, 'speechiness': 0.0752, 'acousticness': 0.705, 'instrumentalness': 3.4e-05, 'liveness': 0.0978, 'valence': 0.735, 'tempo': 118.075, 'type': 'audio_features', 'id': '7oGwQOTkMB9Sk3DIKJLd5F', 'uri': 'spotify:track:7oGwQOTkMB9Sk3DIKJLd5F', 'track_href': 'https://api.spotify.com/v1/tracks/7oGwQOTkMB9Sk3DIKJLd5F', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7oGwQOTkMB9Sk3DIKJLd5F', 'duration_ms': 215787, 'time_signature': 4}]


In [ ]:
def get_rec_params

In [84]:
recs = sp.recommendations(seed_artists=get_rel_arts(art_uri_lst),seed_genres=None, seed_tracks=, limit=100, min_tempo=108,max_tempo=128,min_key=2,max_key=2,min_mode=0,max_mode=0,target_danceability=0.83,target_energy=0.8)
pprint.pprint(recs)


{'seeds': [{'afterFilteringSize': 4,
            'afterRelinkingSize': 4,
            'href': 'https://api.spotify.com/v1/tracks/7oGwQOTkMB9Sk3DIKJLd5F',
            'id': '7oGwQOTkMB9Sk3DIKJLd5F',
            'initialPoolSize': 249,
            'type': 'TRACK'}],
 'tracks': [{'album': {'album_type': 'ALBUM',
                       'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5miWG3FgilzOG7dy3aowZc'},
                                    'href': 'https://api.spotify.com/v1/artists/5miWG3FgilzOG7dy3aowZc',
                                    'id': '5miWG3FgilzOG7dy3aowZc',
                                    'name': 'Thieves Like Us',
                                    'type': 'artist',
                                    'uri': 'spotify:artist:5miWG3FgilzOG7dy3aowZc'}],
                       'available_markets': ['AD',
                                             'AE',
                                             'AL',
                                    

In [58]:
#List of genres accepted by recommendation endpoint
FULL_GENRE_SEED_LST = sp.recommendation_genre_seeds()['genres']

fl = u'\u266D' #flat symbol
sh = u'\u266F' #sharp symbol

#Mapping For Tonics
music_keys = {
    -1: "No key detected {}".format(u'\u2639'),
    0: "C",
    1: "C{}/D{}".format(sh,fl),
    2: "D",
    3: "D{}/E{}".format(sh,fl),
    4: "E",
    5: "F",
    6: "F{}/G{}".format(sh,fl),
    7: "G",
    8: "G/A".format(sh,fl),
    9: "A",    
    10: "A{}/B{}".format(sh,fl),
    11: "B"
}

#Mapping For Modes
mode = {0: 'Min', 1: "Maj"}


PROCEDURAL FLOW:

1. Retrieve all the above info on primary song. 
2. Retrieve related artists first (grab genres here just in case). Retrieve genres of artist(s). Retrieve related artists to the artist(s). Lastly, will need to decide
    between either getting tracks from most popular playlists after getting the genres OR grabbing top tracks
    from related artists.
3. Filter, clean, present nicely. Add option to save as a playlist. 

In [65]:



#we'll use the recommendations endpoint and related artists endpoint

#Spotipy supports this below.
#RECOMMENDATIONS
#parameters to use (consider giving user the option to tweak things like tempo, key)

#limit: will need to be 100 always.
#max_*: example: max_tempo, max_key
#min_*: min_tempo, min_key
#seed_artists, seed_tracks, seed_genres: artists and tracks need to be comma sep list of 
#Spotify IDs. Can use up to 5 values for each.
#target_*: target_danceability, etc. Basically will prioritize these fields in recs it makes.


#using genres as seeds will not work unless explicitly in the full genre seed list for now. 
#Use artists of the song and top related artists for artist seeds.

#RELATED ARTISTS
#if just 1 artist on song: use all the related artists. 
#if more than 2 artists but less than 4: use related artists from first artist in list
#grab genres from related artists and store in list. 

# art_call = sp.artists(art_uri_lst)

# def get_rel_arts(artist_uris):
#     if len(artist_uris) > 4:
#         break
#     else:
#         rel_art_call_data = sp.artist_related_artists(artist_uris[0])
#         for x in rel_art_call_data['artists']:
#             if len(artist_uris) < 5:
#                 artist_uris.append(x['uri'])
#             else:
#                 break
#     return artist_uris



